In [1]:
# !pip install konlpy
# !pip install pandas
# !pip install scikit-learn
# !pip install seaborn

In [2]:
import os 
import json
import pickle
from datetime import datetime
from copy import deepcopy

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Bidirectional, Attention, Concatenate, Dense
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from konlpy.tag import Okt

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x") # svg, retina, png2x ...
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

plt.rc("font", family = "Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False

# 파일 저장시 파일명의 용이성
def now_time():
    now = datetime.now()
    return now.strftime('%Y%m%d_%H_%M_%S')

### 불러오기

In [3]:
base_dir_1 = 'data_prep/data/_2_after_prep/corpus_method_1/'
base_dir_2 = 'data_prep/data/_2_after_prep/corpus_method_2/'


with open(base_dir_1 + "sentences.pkl","rb") as f:
    sentences = pickle.load(f)

with open(base_dir_2 + "labels.pkl","rb") as f:
    labels = pickle.load(f)

with open(base_dir_1 + "sentences_corpus_word_index.json","r") as f:
    sentences_corpus_word_index = json.load(f)

with open(base_dir_2 + "label_corpus_word_index.json","r") as f:
    label_corpus_word_index = json.load(f)

---

### 패딩 작업

In [4]:
# 이렇게 분리 하는 이유 (기존의 트레인 테스트의 tsv를 하나로 합쳤기 때문)
train_sentences, test_sentences = sentences[:15005], sentences[15005:] 
train_labels, test_labels = labels[:15005], labels[15005:]

In [5]:
decoder_input_data = deepcopy(train_labels) # 디코더의 입력 : end 토큰만 제외
target_sequences   = deepcopy(train_labels)   # 타겟 데이터 : 에서 start 토큰을 제외한 값

In [6]:
# 정답 라벨에 각 요소별 시작 1 토큰 추가 및 끝토큰 추가
for i in decoder_input_data:
    i.insert(0,1)
for i in target_sequences:
    i.append(2)

In [7]:
print(decoder_input_data[0])
print(target_sequences[0])

[1, 3]
[3, 2]


- 코퍼스 인덱스 : 단어 역변환 (나중에 찾기 쉽게하기 위함)

In [8]:
sentences_corpus_index_word = {sentences_corpus_word_index[key]:key for key in sentences_corpus_word_index}
label_corpus_index_word = {label_corpus_word_index[key]:key for key in label_corpus_word_index}

- 패딩 작업

In [9]:
max_text_len = max([len(i) for i in train_sentences])
text_padded = pad_sequences(train_sentences, maxlen=max_text_len, padding='post')

max_label_len = max([len(i) for i in decoder_input_data])
decoder_input_data_padded = pad_sequences(decoder_input_data, maxlen=max_label_len, padding='post')
target_sequences_padded = pad_sequences(target_sequences, maxlen=max_label_len, padding='post')

- 전체 코퍼스 크기

In [10]:
corpus_size = len(sentences_corpus_word_index)
corpus_size_label = len(label_corpus_word_index)
corpus_size, corpus_size_label

(38712, 19)

In [11]:
# 불러온 데이터 확인하기
for i in train_sentences[-1]:
    print(sentences_corpus_index_word[i], end=' ')

print("")
for i in train_labels[-1]:
    print(label_corpus_index_word[i])

# 불러온 데이터 확인하기
for i in test_sentences[-1]:
    print(sentences_corpus_index_word[i], end=' ')

print("")
for i in test_labels[-1]:
    print(label_corpus_index_word[i])

멸치 가 틀 딱 짜증나서 트러블 나면 조직 적 으로 좀 스럽게 보복 함 ex : 수건 찜 유도탄 , 틀 한 명 한 명의 살인 광선 , 긴박할 때 말 걸어서 사고 유도 , 좁은 곳 에서 안 비키기 , 샤워 할 때 후 장 씻으며 물 튀기기 
연령
참군 남이노 남자 의 용도 는 고 기 방패 다 ㅋㅋㅋ 
남성


In [12]:
max_label_len, corpus_size_label

(10, 19)

In [13]:
import model._1_lstm_with_attention_model_2_cleaned as model_lstm_att

# best 파라미터로 훈련시켜 최종 모델 제작

In [14]:
max_text_len, corpus_size, max_label_len, corpus_size_label

(77, 38712, 10, 19)

In [18]:
# 실험 1 및 결과
# Embedding1s = [32,64,128,256,512,1024]
# Embedding2s = [32,64,128,256,512,1024]
# lstm_sizes = [32,64,128,256,512,1024]
# dropout_ratios = [0.2,0.4,0.6,0.8]

# 해석 결과 : 임베딩 사이즈를 조금 더 늘려서 실험해보는 작업을 해야 겠다.
#############################################################

# 실험 2 및 결과
# Embedding1s = [2048]
# Embedding2s = [2048]
# lstm_sizes = [64,256,1024,2048]
# dropout_ratios = [0.2,0.4,0.6,0.8] - 컴퓨터의 한계로인해 gpu 다운

# 실험 2 이어서 - 이후 코랩 진행 결과
# Embedding1s = [2048]
# Embedding2s = [2048]
# lstm_sizes = [2048]
# dropout_ratios = [0.2,0.4,0.6,0.8]

# Best
# embedding_size1=2048, embedding_size2=2048, lstm_size=1024, dropout_ratio=0.6 의 결과 최소 loss 값: 0.1358497142791748
#############################################################

# 추가실험 멀티 LSTM 층 추가
# 인코더 디코더에 각 각 총 2층의 LSTM
# 총 학습 파라미터
# Total params: 255,608,851
# 더 나은 결과를 얻지 못함

#############################################################

# 추가실험 멀티 LSTM 층 추가
# 인코더에 총 2 층의 LSTM과 디코더에는 단 1층의 LSTM
# 총 학습 파라미터
# Total params: 166,~~~,~~~
# 더나은 결과를 얻지못함

############################################################
#
# 이후 각 인코더 및 디코더에 dense 층을 늘려보고 정규화 과정을 해보았지만,
# 큰 개선은 없어보임
#
#############################################################
# 결국 각 1층씩의 LSTM

# 최종 파라미터
# Total params: 138,135,571

# embedding1 = 2048
# embedding2 = 2048
# lstm_size = 1024
# dropout_ratio = 0.6
##############################################################

embedding1 = 2048
embedding2 = 2048
lstm_size = 1024
dropout_ratio = 0.6

model, encoder_model, decoder_model = \
model_lstm_att.seq2seq_with_attention(
    max_text_len,
    corpus_size,
    max_label_len,
    corpus_size_label,
    embedding_size1=embedding1,
    embedding_size2=embedding2,
    lstm_size=lstm_size,
    dropout_ratio=dropout_ratio)

model.summary()
# # 모델 학습
check_path = 'checkpoint'
check_path_list = os.listdir(check_path)
# # 학습시

early_stopping = EarlyStopping(monitor='val_loss', patience=1, min_delta=0.001)
checkpoint = ModelCheckpoint(check_path + f'/epoch_'+'{epoch:04d}_metrics_{loss:.4f},{accuracy:.4f},{val_loss:.4f},{val_accuracy:.4f}.h5', save_freq='epoch')
# 모델 학습
# 입력 데이터 : 패딩된 텍스트 시퀀스
# 디코더 인풋데이터 : 끝 토큰을 제외한 값
# 타겟 데이터 : 시작 토큰을 제외한 값

history_model = model.fit([text_padded, decoder_input_data_padded], target_sequences_padded,
                        validation_split=0.1,
                        epochs=1000,
                        initial_epoch=len(check_path_list),
                        batch_size=128,
                        callbacks=[early_stopping,checkpoint],
                        verbose=1)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 77)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 77, 2048)     79282176    ['input_13[0][0]']               
                                                                                                  
 input_14 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 dropout_8 (Dropout)            (None, 77, 2048)     0           ['embedding_4[0][0]']            
                                                                                            

In [19]:
_ = load_model('final_model.h5', compile=False)
model.set_weights(_.get_weights())

In [20]:
sets_for_predict = sentences_corpus_word_index,max_text_len, label_corpus_word_index, label_corpus_index_word, max_label_len

In [21]:
moonjang_ = '안녕 하십니까 반가워요? '
model_lstm_att.predict_from_seq(moonjang_, encoder_model, decoder_model, sets_for_predict)

['clean']

---

### 테스트 데이터를 이용하여 정답 확인

In [22]:
from IPython.display import clear_output

In [23]:
correct = 0
has_more_than_one = 0
sum_pred = 0
predict_labels = []
correct_labels = []

for i in range(len(test_sentences)):
    test_sen = ' '.join([sentences_corpus_index_word[j] for j in test_sentences[i]])
    predict_label = model_lstm_att.predict_from_seq(test_sen, encoder_model, decoder_model, sets_for_predict)
    correct_label = [label_corpus_index_word[k] for k in test_labels[i]]

    # 정확도 계산
    sum_pred += 1
    correct += 1 if sorted(predict_label) == sorted(correct_label) else 0

    # 부분 일치 처리
    has_more_than_one += 1 if set(predict_label).intersection(set(correct_label)) else 0

    predict_labels.append(predict_label)
    correct_labels.append(correct_label)

    # 진행 상황 출력
    clear_output(wait=True)
    print(f'현재 인덱스 진행 ({i + 1}/{len(test_sentences)})')
    print('정답률:', correct / sum_pred * 100)
    print('하나라도 겹치는것이 있는 비율:', has_more_than_one / sum_pred * 100)

    print(f'''
    문제 : {test_sen}
    정답 : {correct_label} | 예측 : {predict_label}
    {('정답!' if predict_label == correct_label else '오답!')}
    ''')
clear_output(wait=True)
print('최종 정확도:', correct / sum_pred * 100)
print('하나라도 겹치는것이 있는 비율:', has_more_than_one / sum_pred * 100)

최종 정확도: 63.25929890286326
하나라도 겹치는것이 있는 비율: 69.44072785656944


### 고찰
학습시 train 데이터에 대한 정확도에 비해 현저히 낮은 정확도의 현상을 보이고있다. <br>
교사 학습으로 인해 학습시에는 좀더 높은 정확도가 나왔을 것이라 판단 된다. <br>

그럼에도 불구하고, 단순 인코더와 dense 층의 모델과 큰 정확도 차이를 보여준다. 아래 코드와 정확도 비교<br>

---

### seq2dense 과의 비교
- valid 데이터에서의 결과 50퍼센트를 넘기지 못함...
```py
for i in np.arange(0.2,0.31,0.001):
    pred = model.predict(X_test,verbose=0) > 0.2

    number_of_count = 0
    correct = 0
    for j in pred == y_test.to_numpy():
        if False not in j:
            correct +=1
        number_of_count +=1
    print(f'트레쉬 홀드가 {i}일때 정확도',correct/number_of_count)

# incoder 와 Dense 층만 있는 모델의 valid 데이터에 대한 정확도
# 트레쉬 홀드가 0.00일때 정확도 0.00
# 트레쉬 홀드가 0.10일때 정확도 0.40
# 트레쉬 홀드가 0.20일때 정확도 0.45
# 트레쉬 홀드가 0.30일때 정확도 0.46
# 트레쉬 홀드가 0.40일때 정확도 0.46
# 트레쉬 홀드가 0.50일때 정확도 0.44
# 트레쉬 홀드가 0.60일때 정확도 0.43
# 트레쉬 홀드가 0.70일때 정확도 0.40
# 트레쉬 홀드가 0.80일때 정확도 0.37
# 트레쉬 홀드가 0.90일때 정확도 0.31
```

### 틀린 문장 확인해보기

In [ ]:
# # 테스트 데이터 number 까지의
# number = 10
# print("완전 정확히 못맞춘 문장들")
# for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
#     if c != p:
#         sen = ''
#         for w in s:
#             sen += sentences_corpus_index_word[w] + ' '
#         print("문장: ",sen)
#         print("정답: ",c,",예측: ",p)
#         print(" ")
        
# print("-"*100)
# print("하나도 못 맞춘 문장들")
# for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
#     if c != p:
#         sen = ''
#         for w in s:
#             sen += sentences_corpus_index_word[w] + ' '
#         if set(c).intersection(set(p)) == set():
#             print("문장: ",sen)
#             print("정답: ",c,",예측: ",p)
#             print(" ")

# print("-"*100)
# print("하나라도 맞춘 문장들")
# for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
#     if c != p:
#         sen = ''
#         for w in s:
#             sen += sentences_corpus_index_word[w] + ' '
#         if set(c).intersection(set(p)) != set():
#             print("문장: ",sen)
#             print("정답: ",c,",예측: ",p)
#             print(" ")
